In [9]:
import tkinter as tk
from tkinter import filedialog, messagebox, Toplevel
from PIL import Image, ImageTk
import cv2
import numpy as np
import imutils
import easyocr
from datetime import datetime

def quit():
    try:
        window.destroy()
    except tk.TclError:
        pass

# Sample plates dictionary for testing
plates_dict = {
    "ABC1234": {"time": "10:00:00"},
    "XYZ5678": {"time": "11:00:00"},
    "LMN2345": {"time": "12:00:00"}
}

def ticket():
    plate_number = entry1.get().strip().upper()
    
    if plate_number in plates_dict:
        time_of_plate = plates_dict[plate_number]["time"]
        time_format = "%H:%M:%S"
        time_scanned = datetime.strptime(time_of_plate, time_format)
        current_time = datetime.now()
        time_taken_minutes = (current_time - time_scanned).seconds // 60  # time taken in minutes excluding seconds
        price = time_taken_minutes * 10
        
        # Create a new window to display the information
        info_window = Toplevel()
        info_window.title("Plate Information")
        info_window.geometry("300x200")
        
        plate_label = tk.Label(info_window, text=f"Plate Number: {plate_number}", font=("Arial", 12))
        plate_label.pack(pady=10)
        
        time_label = tk.Label(info_window, text=f"Time Scanned: {time_of_plate}", font=("Arial", 12))
        time_label.pack(pady=10)
        
        time_taken_label = tk.Label(info_window, text=f"Time Taken: {time_taken_minutes} minutes", font=("Arial", 12))
        time_taken_label.pack(pady=10)
        
        price_label = tk.Label(info_window, text=f"Price: ${price:.2f}", font=("Arial", 12))
        price_label.pack(pady=10)
        
    else:
        messagebox.showinfo("Error", "Plate not found.")

def login():
    global file_path  # Declare the global variable
    global scanned_plate  # Declare a global variable to store the scanned plate text
    global scan_time  # Declare a global variable to store the scan time
    file_path = None  # Initialize it as None
    scanned_plate = None  # Initialize it as None
    scan_time = None  # Initialize it as None

    username = "come"
    password = "436"

    if username_entry.get() == username and password_entry.get() == password:
        window.destroy()
        
        new_window = tk.Tk()
        new_window.title("Plate Recognition System")
        new_window.geometry('800x500')
        new_window.configure(bg='#80868a')

        frame = tk.Frame(new_window)
        
        initial_photo = Image.open("dft.png")
        resized_initial_image = initial_photo.resize((450, 250), Image.Resampling.LANCZOS)
        converted_initial_image = ImageTk.PhotoImage(resized_initial_image)
        
        global entry1
        entry1 = tk.Entry(new_window)
        entry1.place(x=570, y=70, height=35, width=180)

        def search_plate():
            plate_number = entry1.get().strip().upper()
            if plate_number in plates_dict:
                time_of_plate = plates_dict[plate_number]["time"]
                time_format = "%H:%M:%S"
                time_scanned = datetime.strptime(time_of_plate, time_format)
                current_time = datetime.now()
                time_taken_minutes = (current_time - time_scanned).seconds // 60  # time taken in minutes excluding seconds
                
                result = (f"Plate Number: {plate_number}\n"
                          f"Time Scanned: {time_of_plate}\n"
                          f"Time Taken: {time_taken_minutes} minutes")
            else:
                result = "Plate not found."
            messagebox.showinfo("Search Result", result)

        def upload():
            global file_path  # Use the global variable
            file_path = filedialog.askopenfilename()
            if file_path:
                label.config(text=f"Selected File: {file_path}")
                uploaded_photo = Image.open(file_path)
                resized_uploaded_image = uploaded_photo.resize((450, 250), Image.Resampling.LANCZOS)
                converted_uploaded_image = ImageTk.PhotoImage(resized_uploaded_image)
                image_label.config(image=converted_uploaded_image)
                image_label.image = converted_uploaded_image
            else:
                label.config(text="No file selected.")

        def scan():
            global file_path  # Use the global variable
            global scanned_plate  # Use the global variable to store the scanned plate text
            global scan_time  # Use the global variable to store the scan time
            if file_path is None:
                messagebox.showerror("Error", "No file selected. Please upload an image first.")
                return

            image = cv2.imread(file_path)
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            bfilter = cv2.bilateralFilter(gray_image, 11, 17, 17)
            edged = cv2.Canny(bfilter, 30, 200)
            keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            contours = imutils.grab_contours(keypoints)
            contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]
            location = None
            for contour in contours:
                approx = cv2.approxPolyDP(contour, 10, True)
                if len(approx) == 4:
                    location = approx
                    break

            if location is not None:
                mask = np.zeros(gray_image.shape, np.uint8)
                cv2.drawContours(mask, [location], 0, 255, -1)
                masked_image = cv2.bitwise_and(image, image, mask=mask)
                (x, y) = np.where(mask == 255)
                (x1, y1) = (np.min(x), np.min(y))
                (x2, y2) = (np.max(x), np.max(y))
                cropped_image = gray_image[x1:x2+1, y1:y2+1]
                
                zoom_factor = 2
                zoomed_image = cv2.resize(cropped_image, None, fx=zoom_factor, fy=zoom_factor, interpolation=cv2.INTER_LINEAR)
                cv2.imwrite('zoomed_image.jpg', zoomed_image)
                
                zoomed_image_rgb = cv2.cvtColor(zoomed_image, cv2.COLOR_BGR2RGB)
            else:
                print("No contour with 4 vertices found.")
            
            reader = easyocr.Reader(['en'])
            result = reader.readtext(zoomed_image)
            print(result)
            
            if result:
                scanned_plate = result[0][-2]  # Assuming the first result contains the plate text
                scan_time = datetime.now()  # Get the current time as a datetime object
                messagebox.showinfo("Scanned Plate", f"Scanned Plate: {scanned_plate} at {scan_time.strftime('%H:%M:%S')}")
            else:
                scanned_plate = None
                scan_time = None
                messagebox.showerror("Error", "No text detected in the image.")

        def add_to_dict():
            global scanned_plate  # Use the global variable
            global scan_time  # Use the global variable
            if scanned_plate and scan_time:
                plates_dict[scanned_plate] = {"time": scan_time.strftime('%H:%M:%S')}  # Add the scanned plate with the scan time
                messagebox.showinfo("Success", f"Plate {scanned_plate} added to the dictionary with time {scan_time.strftime('%H:%M:%S')}.")
            else:
                messagebox.showerror("Error", "No plate scanned to add.")

        btn_search = tk.Button(new_window, text="SEARCH", width=16, height=1, command=search_plate)
        btn_search.place(x=600, y=135)
        btn_delete = tk.Button(new_window, text="TICKET", width=16, height=1, command=ticket)
        btn_delete.place(x=600, y=215)
        
        btn_info = tk.Button(new_window, text="RECORD", width=16, height=1, command=add_to_dict)
        btn_info.place(x=600, y=175)
        
        btn_upload = tk.Button(new_window, text="UPLOAD", width=20, height=3, command=upload, bg='#dbc05e')
        btn_upload.place(x=50, y=370)
        
        btn_scan = tk.Button(new_window, text="SCAN", width=20, height=3, command=scan, bg='#5e94db')
        btn_scan.place(x=300, y=370)
        
        btn_quit = tk.Button(new_window, text="QUIT", width=16, height=1, command=new_window.destroy, bg='#db5e77')
        btn_quit.place(x=600, y=400)

        image_label = tk.Label(new_window, image=converted_initial_image, width=450, height=250)
        image_label.image = converted_initial_image
        image_label.grid(row=4, column=8, columnspan=2, pady=30, padx=30)

        label = tk.Label(new_window, text="No file selected.", bg='#80868a', fg='#FFFFFF')
        label.place(x=50, y=290)

        new_window.mainloop()
    else:
        messagebox.showinfo(title="Invalid Login", message="Invalid login")

window = tk.Tk()
window.geometry('800x500')
window.title('System Login')
window.configure(bg='#80868a')

frame = tk.Frame(window, bg='#80868a')

sys_label = tk.Label(frame, text="Plate Recognition System", bg='#80868a', fg='#FFFFFF', font=("Arial", 20))
login_label = tk.Label(frame, text="Login", bg='#80868a', fg='#FFFFFF', font=("Arial", 30))
username_label = tk.Label(frame, text="Username", bg='#80868a', fg='#FFFFFF')
username_entry = tk.Entry(frame, bg='#e9f2f2')
password_entry = tk.Entry(frame, show="*", bg='#e9f2f2')
password_label = tk.Label(frame, text="Password", bg='#80868a', fg='#FFFFFF')
login_button = tk.Button(frame, text="Login", bg='#5edba9', fg='#FFFFFF', command=login, borderwidth=1, width=20, height=2)
quit_button = tk.Button(frame, text="Quit", bg='#db5e77', fg='#FFFFFF', command=quit, borderwidth=1, width=20, height=1)

image = Image.open("dou.png")
res_image = image.resize((150, 150), Image.Resampling.LANCZOS)
conv_img = ImageTk.PhotoImage(res_image)

sys_label.grid(row=1, column=0, columnspan=2)
login_label.grid(row=0, column=0, columnspan=2, sticky="news", pady=40)
username_label.grid(row=2, column=0, padx=1)
username_entry.grid(row=2, column=1, pady=10)
password_entry.grid(row=3, column=1, pady=0)
password_label.grid(row=3, column=0)
login_button.grid(row=4, column=0, columnspan=2, pady=40)
quit_button.grid(row=7, column=0, columnspan=2, pady=70)
logo_label = tk.Label(frame, image=conv_img, width=150, height=150, bg='#80868a', fg='#FFFFFF')
logo_label.grid(row=0, column=0, columnspan=2, pady=0)

frame.pack()

window.mainloop()